In [55]:
import socks
import socket
import requests
from bs4 import BeautifulSoup
import time
import re
import numpy as np
import pandas as pd

socks.set_default_proxy(socks.SOCKS5, "localhost", 9050) # one source said port 9150, Georgia's code used 9050
socket.socket = socks.socksocket

# These lines are copied from the Linkedin post.
def getaddrinfo(*args):
    return [(socket.AF_INET, socket.SOCK_STREAM, 6, '', (args[0], args[1]))]
socket.getaddrinfo = getaddrinfo

# ToRReZ market URL DEZE LINKE GEBRUIKEN OM DE COOCKIES TE PAKKEN
url = "http://yxuy5oau7nugw4kpb4lclrqdbixp3wvc4iuiad23ebyp2q3gx7rtrgqd.onion/items/category/drugs-and-chemicals?order=none&vendors=all&shipping_to=any&shipping_from=any&product_type=any&payments_method=any&page=1"


# Manually selected from the browser header after solving the captcha
cookies_copied = 'Cookie: ray_id=f44367b500980adfa0bba58b912fc065f3328016277fdfeb54120ede4bbc038a; hr4ujvby8ds459og4kzcpbzwjdj_session=eyJpdiI6ImR1R1pPUWI4YndwOXpcL1ZlQVJlU2t3PT0iLCJ2YWx1ZSI6IjZXWHhtZ3pxU3U3TVE3OFBuY2JiYzdTaExUKzdRZ2ZaQ0dRQml3MkVXN3JCRzR5K3IxN083NDJ5SUFONm84c2FyNE1GMWJBa25tb05Ib1VCQ1wvSWFsVENjUXBsRnFDVzlFVFwvdjNNWEd3ZG1WTzBwbGp3N3BVNXl4SzlScVp1bnciLCJtYWMiOiJlZTU1NjEzZjg0NWVjZjMzMzFlYWNkMzRmZWMxNmQ4NTE4ZjY1NWZkZTM2ZTFmZWMzNTZmM2ZkNjE4MjMxOGM2In0%3D; XSRF-TOKEN=eyJpdiI6InNcL2NHTERFOHZmTjl3bllETUdRV3pRPT0iLCJ2YWx1ZSI6ImplRDM1QnVURVUyRmN3VzN3K1BTSlhqem1EOE53dTRIVGhGNWRvZ3pINHlFXC9SNE9BSG5iV0h4UDdzbjdKQWRJYjNDXC9temFmeDBMVjBzK1wvN3lcLzBaZHlmM1NZU2RTZ0tnelZzMHp4RXF6SkdVYkd2SThwNHJTcU0wQWlsdmw2ZSIsIm1hYyI6IjE5MGMxMDM4NzQxYmM3NDQzNDUxNTU4ZGFhNGI0NzA0YmM3YmVkNTcwNDFlMzczNTU0YzkyYjg1ZmE3MjI0ZDUifQ%3D%3D'


# Takes the relevant info of the copied cookies and puts it in the correct format automatically.
def convert_pasted_cookies_to_usable_format(pasted_cookies):
    all_cookies_str = pasted_cookies.split(':')[1] # to ignore the prefix "Cookie: ")
    individual_cookies = all_cookies_str.split(';')
    cookies_set = {}
    for cookie in individual_cookies:
        cookies_set.update({cookie.strip().split('=')[0]: cookie.strip().split('=')[1]})
    return cookies_set


s = requests.Session()
s.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0'
})

proxies = {
    'http': 'socks5h://localhost:9050',
    'https': 'socks5h://localhost:9050'
}

set_of_cookies = convert_pasted_cookies_to_usable_format(cookies_copied)

# Get the page category drugs
web_page = s.get(url, cookies=set_of_cookies, proxies=proxies)
soup = BeautifulSoup(web_page.content, 'html.parser')



# create empty lists
vendor = []    
vendor_offers = []
all_titles = []
all_quantities = []
all_prices = []
category = []
category_level_1 = []
category_level_2 = []
category_level_3 = []
ship_from = []
ship_to =[]



# crawl and scrape through pages
pages = []
for a in soup.find_all('a',attrs={"class" : "page-link"}):
    pages.append(a.text)
last_page = pages[-2]

for page in range(1,int(last_page)+1): #last_page+1

    # NIET AANPASSEN
    url = 'http://yxuy5oau7nugw4kpb4lclrqdbixp3wvc4iuiad23ebyp2q3gx7rtrgqd.onion/items/category/drugs-and-chemicals?order=none&vendors=all&shipping_to=any&shipping_from=any&product_type=any&payments_method=any&page=' + str(page)

    web_page = s.get(url, cookies=set_of_cookies, proxies=proxies)
    soup = BeautifulSoup(web_page.content, 'html.parser')


    for a in soup.find_all('a', href=True, class_='title'):
        all_titles.append(a.text.strip())
        if "gram" in a.text.strip().lower():
            #print(a.text.strip())
            if "." in a.text.strip().lower().split('gram')[0]:
                quantity = (re.findall("\d+\.\d+", a.text.strip().lower().split('gram')[0]))[0]
            else:
                quantity = (re.findall("\d+", a.text.strip().lower().split('gram')[0]))[0]
        else:
            #print("No quantity found in: " + a.text.strip())
            quantity = 0

        all_quantities.append(quantity)

    for price in soup.find_all('span', class_='Price'):
        all_prices.append(re.findall("\d+\.\d+", price.text.strip())[0])
        
    for a in soup.find_all('div',attrs={"class" : "w-100 mt-3 small"}):
        if len(a.find_all('a'))==4:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(a.find_all('a')[2].text.strip())
            category_level_3.append(a.find_all('a')[3].text.strip())
        if len(a.find_all('a'))==3:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(a.find_all('a')[2].text.strip())
            category_level_3.append(np.nan)
        if len(a.find_all('a'))==2:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(np.nan)
            category_level_3.append(np.nan)
        last_item = a.find_all('a')[-1]
        category.append(last_item.text.strip())

           
    for tag_elm in soup.find_all('div', class_='shipping'):
        if tag_elm.find('span', class_='shippingFrom') != None:
            ship_from.append(tag_elm.find('span', class_='shippingFrom').text.strip())
        else:
            ship_from.append(np.nan)

    for tag_elm in soup.find_all('div', class_='shipping'):
        if tag_elm.find('span', class_='shippingTo') != None:
            ship_to.append(tag_elm.find('span', class_='shippingTo').text.strip())
        else:
            ship_to.append(np.nan)

    for a in soup.find_all('a', class_='font-weight-bold'):
        vendor.append(a.text.strip())


            
            
results = (list(zip(vendor, category_level_1, category_level_2, category_level_3, category, all_titles, all_quantities, all_prices, ship_from, ship_to)))



df = pd.DataFrame(results,columns=['vendor', 'category_level_1', 'category_level_2', 'category_level_3', 'highest_category', 'product','quantity','price in $', 'shipping_from', 'shipping_to'])    


In [56]:
df

,vendor,category_level_1,category_level_2,category_level_3,highest_category,product,quantity,price in $,shipping_from,shipping_to
0,CornerShop,Drugs and Chemicals,NaN,NaN,Drugs and Chemicals,Fentanyl Pflaster abgelaufen,0,42.17,Germany,Europe
1,GQDS24,Drugs and Chemicals,Cannabis & Hash,Hash,Hash,5 Gramm Pollum Hasch A+++,5,60.24,Germany,Germany
2,GQDS24,Drugs and Chemicals,Cannabis & Hash,Buds & Flowers,Buds & Flowers,20 Gramm Super Lemon Haze A+++,20,240.96,Germany,Germany
3,GQDS24,Drugs and Chemicals,Cannabis & Hash,Buds & Flowers,Buds & Flowers,10 Gramm Super Lemon Haze A+++,10,132.53,Germany,Germany
4,GQDS24,Drugs and Chemicals,Cannabis & Hash,Buds & Flowers,Buds & Flowers,5 Gramm Super Lemon Haze A+++,5,72.29,Germany,Germany
5,GQDS24,Drugs and Chemicals,Cannabis & Hash,Hash,Hash,20 Gramm Pollum Hasch A+++,20,192.77,Germany,Germany
6,GQDS24,Drugs and Chemicals,Cannabis & Hash,Hash,Hash,10 Gramm Pollum Hasch A+++,10,108.43,Germany,Germany
